In [1]:
import json
import nltk
import string
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:
intents = json.loads(open('intents.json').read())
words=[]
classes = []
documents = []
ignore_words = string.punctuation

In [13]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w=nltk.word_tokenize(pattern)
        words.extend(w)        
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [16]:
# lemmaztize and lower each word and remove duplicates using set
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [30]:
# create training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [18]:
documents[:10]

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye')]

In [32]:
#create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [33]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-33-9ffac2c3f218>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [37]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [38]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [39]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
25/25 [==============================] - 2s 2ms/step - loss: 3.7796 - accuracy: 0.0000e+00
Epoch 2/200
25/25 [==============================] - 0s 1ms/step - loss: 3.7412 - accuracy: 0.0374
Epoch 3/200
25/25 [==============================] - 0s 1ms/step - loss: 3.6350 - accuracy: 0.1050
Epoch 4/200
25/25 [==============================] - 0s 1ms/step - loss: 3.5104 - accuracy: 0.1170
Epoch 5/200
25/25 [==============================] - 0s 1ms/step - loss: 3.5532 - accuracy: 0.1041
Epoch 6/200
25/25 [==============================] - 0s 1ms/step - loss: 3.2833 - accuracy: 0.1634
Epoch 7/200
25/25 [==============================] - 0s 1ms/step - loss: 3.2299 - accuracy: 0.1271
Epoch 8/200
25/25 [==============================] - 0s 1ms/step - loss: 3.1367 - accuracy: 0.1125
Epoch 9/200
25/25 [==============================] - 0s 1ms/step - loss: 2.8804 - accuracy: 0.2423
Epoch 10/200
25/25 [==============================] - 0s 1ms/step - loss: 2.8101 - accuracy: 0.3288
Epoch

Epoch 83/200
25/25 [==============================] - 0s 1ms/step - loss: 0.2075 - accuracy: 0.9549
Epoch 84/200
25/25 [==============================] - 0s 1ms/step - loss: 0.4421 - accuracy: 0.8841
Epoch 85/200
25/25 [==============================] - 0s 1ms/step - loss: 0.4975 - accuracy: 0.8786
Epoch 86/200
25/25 [==============================] - 0s 1ms/step - loss: 0.4214 - accuracy: 0.8308
Epoch 87/200
25/25 [==============================] - 0s 1ms/step - loss: 0.5173 - accuracy: 0.8893
Epoch 88/200
25/25 [==============================] - 0s 1ms/step - loss: 0.2229 - accuracy: 0.9337
Epoch 89/200
25/25 [==============================] - 0s 1ms/step - loss: 0.2444 - accuracy: 0.9371
Epoch 90/200
25/25 [==============================] - 0s 1ms/step - loss: 0.2917 - accuracy: 0.9345
Epoch 91/200
25/25 [==============================] - 0s 1ms/step - loss: 0.2583 - accuracy: 0.9354
Epoch 92/200
25/25 [==============================] - 0s 1ms/step - loss: 0.3194 - accuracy: 0.8820


In [40]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [41]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [44]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    print(res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [45]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

In [47]:
w=predict_class('appoitment with',model)

[1.4553943e-03 4.3148870e-05 3.6457344e-04 4.1711781e-05 2.8809786e-02
 8.7379674e-05 4.0969398e-06 4.3401378e-06 9.0012521e-01 1.0704951e-03
 9.6301977e-05 2.1867421e-03 6.4575273e-05 3.7518272e-04 1.9989708e-05
 5.2132818e-05 1.8066205e-05 5.4197353e-05 3.0680976e-04 2.7208020e-05
 1.3493362e-05 2.2263548e-04 1.2854524e-07 1.2651456e-05 8.4096177e-07
 2.6558313e-05 1.0626557e-06 2.1321652e-05 1.8715188e-03 2.9656730e-06
 1.0199985e-06 3.0148434e-02 3.2338686e-02 1.7505899e-06 1.8936398e-05
 3.5616409e-05 4.7272311e-05 1.3574934e-05 3.2262581e-06 8.6257178e-06
 8.7390376e-08 1.0701775e-06 1.1660431e-06]


In [58]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))